# Load Libaries

In [1]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import glob
import pickle
import warnings
from scipy import misc
import scipy.ndimage
from skimage import feature
from PIL import Image
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.models import model_from_json
from keras.callbacks import EarlyStopping
from keras import backend as K

Using TensorFlow backend.


# Load training and test data

In [2]:
def get_train_and_test_data(procent, model_number, augmented_data_list):
    train_avo_image_folder = "avo_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train_images/"
    train_avo_labels_folder = "avo_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train_labels/"
    
    test_avo_image_folder = "final_data_avo/data_" + str(procent) + "/model_" + str(model_number) + "/test_images/"
    test_avo_labels_folder = "final_data_avo/data_" + str(procent) + "/model_" + str(model_number) + "/test_labels/"

    train_X = []
    train_Y = []
    test_X = []
    test_Y = []

    for filename in os.listdir(train_avo_image_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_avo_image_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_X.append(img)
            label = np.loadtxt(train_avo_labels_folder + filename[:-4] + "_label.txt", dtype=int)
            train_Y.append(label)

    for filename in os.listdir(test_avo_image_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(test_avo_image_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            test_X.append(img)
            label = np.loadtxt(test_avo_labels_folder + filename[:-4] + "_label.txt", dtype=int)
            test_Y.append(label)

            
    # Augmented training data
    for augmentation in augmented_data_list:
        augmented_train_image_folder = "final_data_avo/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "_images/"
        augmented_train_label_folder = "final_data_avo/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "_labels/"
        
        for filename in os.listdir(augmented_train_image_folder):
            if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                if augmentation == "final_vae":
                    img = (cv2.imread(augmented_train_image_folder + filename, cv2.IMREAD_GRAYSCALE))
                else:
                    img = (cv2.imread(augmented_train_image_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                train_X.append(img)
                label = np.loadtxt(augmented_train_label_folder + filename[:-4] + "_label.txt", dtype=int)
                train_Y.append(label)



    # Convert lists to np array
    np_train_images = np.array(train_X)

    # Convert lists to np array
    np_test_images = np.array(test_X)

    # Data labels
    train_labels = np.array(train_Y)

    test_labels = np.array(test_Y)

    train_data = np_train_images.reshape(np_train_images.shape[0], 128, 128, 1)
    test_data = np_test_images.reshape(np_test_images.shape[0], 128, 128, 1)
    return train_data, train_labels, test_data, test_labels

# Define Loss function

In [4]:
def differnce(y_true, y_pred):
    return K.mean(np.abs(y_true - y_pred))

In [3]:
def run_model(train_data, train_labels, test_data, test_labels, patience):
    
    # Model
    model = Sequential()
    model.add(Conv2D(8, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu, input_shape=(128,128,1)))
    model.add(Conv2D(8, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu))
    model.add(Conv2D(8, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu))
    model.add(Flatten())
    model.add(Dense(1024, activation=tf.nn.relu))
    model.add(Dense(512, activation=tf.nn.relu))
    model.add(Dense(1, activation="linear"))
    
    
    # Compile Model
    model.compile(optimizer='adam', 
                  loss=differnce,
                  metrics=[differnce])

    
    # Define Earlystopping
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)


    # Get history
    history = model.fit(x=train_data,y=train_labels, validation_data=(test_data, test_labels), shuffle=True, batch_size=32, epochs=1000, verbose=0, callbacks=[es])
    
    
    # Get loss
    history_loss = np.array(history.history['loss'])
    history_val_loss = np.array(history.history['val_loss'])

    _, train_difference = model.evaluate(train_data, train_labels, verbose=0)
    _, test_difference = model.evaluate(test_data, test_labels, verbose=0)
    
    # Clear meomory
    K.clear_session()
    return train_difference, test_difference, np.mean(history_loss[-30:]), np.mean(history_val_loss[-30:])


In [7]:
# Write name of augmentations methods
augmentaion_list = []

for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    final_loss = []
    model_loss = 0
    model_acc = 0
    for j in range(5):
        model_acc = 0
        model_loss = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j, augmentaion_list)
        for i in range(5):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            train_difference, test_difference, loss, val_loss  = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_difference, test_difference))
            print("Run {} with model {} got train loss {} and val_loss {}".format(i, j + 1, loss, val_loss))
            
            model_acc += test_difference
            model_loss += val_loss
            
        final_acc.append(model_acc)
        final_loss.append(model_loss)
        
    for i in range(5):
        print("Model {} got an difference on: {}".format(i, final_acc[i] / 5.0))
        print("Model {} got an val loss on: {}".format(i, final_acc[i] / 5.0))
        
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 25))
    print("Using {} procent of data to training our model got an loss {}".format(p, sum(final_loss) / 25))
    end = time.time()
    print(end - start)


KeyboardInterrupt: 